In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [2]:
os.chdir('/home/witoslaw/diffTSS/trained_models/2025-07-17-17')

In [3]:
os.getcwd()

'/home/witoslaw/diffTSS/trained_models/2025-07-17-17'

In [4]:
expr_df = pd.read_csv('/home/witoslaw/data/diffTSS/RNA_CAGE.txt', sep='\t')

In [5]:
expr_df#[expr_df['ENSID'] == 'ENSG00000160072']#['GM12878_CAGE_128*3_sum'].dropna()

,ENSID,K562_CAGE_128*3_sum,GM12878_CAGE_128*3_sum,EXPRESSION,UTR5LEN,CDSLEN,INTRONLEN,UTR3LEN,UTR5GC,UTR3GC,ORFEXONDENSITY,ENSID_old
0,ENSG00000000003,10.621,9.7130,0.267,111,728,5943,1354,0.625,0.368,9.62,ENSG00000000003
1,ENSG00000000003_1,NaN,NaN,0.267,111,728,5943,1354,0.625,0.368,9.62,ENSG00000000003
2,ENSG00000000005,NaN,2.0014,0.006,216,944,13756,167,0.470,0.393,7.42,ENSG00000000005
3,ENSG00000000005_1,NaN,NaN,0.006,216,944,13756,167,0.470,0.393,7.42,ENSG00000000005
4,ENSG00000000419,1506.270,1730.0100,73.719,31,851,22542,264,0.000,0.262,11.75,ENSG00000000419
...,...,...,...,...,...,...,...,...,...,...,...,...
29031,ENSG00000310521,30.212,NaN,0.000,0,0,0,0,0.000,0.000,0.00,0
29032,ENSG00000310523,96.939,26.3820,0.000,0,0,0,0,0.000,0.000,0.00,0
29033,ENSG00000310523_1,NaN,NaN,0.000,0,0,0,0,0.000,0.000,0.00,0
29034,ENSG00000310526,NaN,NaN,0.000,0,0,0,0,0.000,0.000,0.00,0


In [6]:
expr_df_K5 = expr_df[['ENSID', 'K562_CAGE_128*3_sum']]
expr_df_GM = expr_df[['ENSID', 'GM12878_CAGE_128*3_sum']]
print(expr_df_K5)
print(expr_df_GM)

                   ENSID  K562_CAGE_128*3_sum
0        ENSG00000000003               10.621
1      ENSG00000000003_1                  NaN
2        ENSG00000000005                  NaN
3      ENSG00000000005_1                  NaN
4        ENSG00000000419             1506.270
...                  ...                  ...
29031    ENSG00000310521               30.212
29032    ENSG00000310523               96.939
29033  ENSG00000310523_1                  NaN
29034    ENSG00000310526                  NaN
29035    ENSG00000310537                  NaN

[29036 rows x 2 columns]
                   ENSID  GM12878_CAGE_128*3_sum
0        ENSG00000000003                  9.7130
1      ENSG00000000003_1                     NaN
2        ENSG00000000005                  2.0014
3      ENSG00000000005_1                     NaN
4        ENSG00000000419               1730.0100
...                  ...                     ...
29031    ENSG00000310521                     NaN
29032    ENSG00000310523      

In [7]:
results_dict = {}
results_dict['K562'] = pd.DataFrame()
results_dict['GM12878'] = pd.DataFrame()

column_names = ['ENSID', 'Pred', 'actual', 'fold_idx']

for file in [file for file in os.listdir() if 'csv' in file]:
    #print(f"File: {'.'.join(file.split('.')[-5:-2])}")
    if 'K562' in file:
        results_dict['K562'] = pd.concat([results_dict['K562'], pd.read_csv(file, names=column_names, header=0)], ignore_index=True)
    else:
        results_dict['GM12878'] = pd.concat([results_dict['GM12878'], pd.read_csv(file, names=column_names, header=0)], ignore_index=True)

results_dict['K562'] = results_dict['K562'].merge(expr_df_K5, on='ENSID')
results_dict['GM12878'] = results_dict['GM12878'].merge(expr_df_GM, on='ENSID')

for key in results_dict.keys():
    results_dict[key]['error'] = abs(results_dict[key]['Pred'] - results_dict[key]['actual'])

In [8]:
results_dict

{'K562':                    ENSID      Pred    actual  fold_idx  K562_CAGE_128*3_sum  \
 0      ENSG00000143344_1  1.287776  1.463296         1               28.060   
 1        ENSG00000162711  0.321398  0.000000         1                  NaN   
 2        ENSG00000041988  1.514063  2.081203         1              119.560   
 3        ENSG00000289881  1.862921  1.274273         1               17.805   
 4        ENSG00000143195  1.402088  1.801925         1               62.376   
 ...                  ...       ...       ...       ...                  ...   
 28352  ENSG00000138835_3  0.289529  1.421834        12               25.414   
 28353  ENSG00000165181_1  0.113323  0.000000        12                  NaN   
 28354    ENSG00000141179  2.289973  2.415207        12              259.140   
 28355    ENSG00000108379  1.493031  1.106803        12               11.788   
 28356  ENSG00000141068_2  0.343771  0.000000        12                  NaN   
 
           error  
 0      0.1

In [9]:
K562_h5 = h5py.File('/home/witoslaw/data/diffTSS/K562_enhancer_promoter_encoding.rna_encoding.hg38.h5')
GM12878_h5 = h5py.File('/home/witoslaw/data/diffTSS/GM12878_enhancer_promoter_encoding.rna_encoding.hg38.h5')

In [10]:
K562_h5.keys()

<KeysViewHDF5 ['activity', 'distance', 'ensid', 'hic', 'pe_code', 'rna']>

---
# Getting RNA signals for TSSs predicted as negative values

In [15]:
neg_pred_dict = {}
neg_pred_dict['K562'] = []
neg_pred_dict['GM12878'] = []

for key in results_dict.keys():
    #results_dict[key]['error'] = abs(results_dict[key]['Pred'] - results_dict[key]['actual'])
    results_dict[key]['neg_pred'] = results_dict[key]['Pred'] < 0
    neg_pred_dict[key] = list(results_dict[key][results_dict[key]['neg_pred']].iloc[:,0])
    print(f"For {key}")
    print(f"Mean error: {np.mean(results_dict[key]['error'])}")
    print(f"Max error: {np.max(results_dict[key]['error'])}")
    print(f"Min error: {np.min(results_dict[key]['error'])}")
    print(results_dict[key][results_dict[key]['neg_pred']])

For K562
Mean error: 0.4940765429518559
Max error: 4.908513902
Min error: 1.8000000000073513e-05
                   ENSID      Pred  actual  fold_idx  K562_CAGE_128*3_sum  \
6        ENSG00000116183 -0.002844     0.0         1                  NaN   
1829     ENSG00000234754 -0.004653     0.0         1                  NaN   
2111     ENSG00000007933 -0.002311     0.0         1                  NaN   
7317     ENSG00000134489 -0.004991     0.0         4                  NaN   
7458     ENSG00000178522 -0.001096     0.0         4                  NaN   
...                  ...       ...     ...       ...                  ...   
25553    ENSG00000280809 -0.023375     0.0        11                  NaN   
25560    ENSG00000237797 -0.003499     0.0        11                  NaN   
25576  ENSG00000159398_1 -0.012483     0.0        11                  NaN   
25635    ENSG00000260314 -0.008390     0.0        11                  NaN   
25637  ENSG00000151474_2 -0.003149     0.0        11    

In [28]:
np.min(results_dict['K562']['Pred'])

-0.1281749

In [29]:
k5_dict = {}
k5_bool = [gene.decode() in neg_pred_dict['K562'] for gene in K562_h5['ensid']]
k5_dict['rna_signals'] = K562_h5['rna'][:][k5_bool]
k5_dict['gene_labels'] = K562_h5['ensid'][:][k5_bool]

gm_dict = {}
gm_bool = [gene.decode() in neg_pred_dict['GM12878'] for gene in K562_h5['ensid']]
gm_dict['rna_signals'] = GM12878_h5['rna'][:][gm_bool]
gm_dict['gene_labels'] = GM12878_h5['ensid'][:][gm_bool]

In [ ]:
for cell_dict in [k5_dict]:
    for i, val in enumerate(cell_dict['gene_labels']):
        plt.plot(cell_dict['rna_signals'][i])
        plt.title(val.decode())
        plt.show()
        plt.clf()
        print(f"Pred: {results_dict['K562'][results_dict['K562'].iloc[:,0] == val.decode()]['Pred'].iloc[0]}")
        print(f"Actual: {results_dict['K562'][results_dict['K562'].iloc[:,0] == val.decode()]['actual'].iloc[0]}")

---
# Getting RNA signals for TSSs where error < 0.01 

In [183]:
low_error_dict = {}
low_error_dict['K562'] = []
low_error_dict['GM12878'] = []

for key in results_dict.keys():
    results_dict[key]['low_error'] = results_dict[key]['error'] < .01
    low_error_dict[key] = list(results_dict[key][results_dict[key]['low_error']].iloc[:,0])
    print(results_dict[key][results_dict[key]['low_error']])

              Unnamed: 0      Pred    actual  fold_idx     error  neg_pred  \
6        ENSG00000116183 -0.002844  0.000000         1  0.002844      True   
326    ENSG00000174606_1  2.167054  2.168792         1  0.001738     False   
335      ENSG00000271252  0.006273  0.000000         1  0.006273     False   
337    ENSG00000082512_1  1.676264  1.670793         1  0.005471     False   
534    ENSG00000120334_1  2.910146  2.917038         1  0.006892     False   
...                  ...       ...       ...       ...       ...       ...   
27629  ENSG00000107104_1  1.762994  1.771440        12  0.008447     False   
27642    ENSG00000233016  2.703006  2.694162        12  0.008844     False   
27978    ENSG00000189159  1.041013  1.034388        12  0.006625     False   
28155    ENSG00000157637  1.204168  1.198575        12  0.005593     False   
28191  ENSG00000141576_1  2.071029  2.061901        12  0.009128     False   

       low_error  
6           True  
326         True  
335   

In [1]:
k5_bool = [gene.decode() in low_error_dict['K562'] for gene in K562_h5['ensid']]
k5_rna_signals = K562_h5['rna'][:][k5_bool]

gm_bool = [gene.decode() in low_error_dict['GM12878'] for gene in K562_h5['ensid']]
gm_rna_signals = GM12878_h5['rna'][:][gm_bool]

NameError: name 'K562_h5' is not defined

In [ ]:
for signal in gm_rna_signals:
    plt.plot(signal)
    plt.show()

---
# Getting RNA signals for TSSs where error > 3

In [11]:
high_error_dict = {}min
high_error_dict['K562'] = []
high_error_dict['GM12878'] = []

for key in results_dict.keys():
    results_dict[key]['high_error'] = results_dict[key]['error'] > 3
    high_error_dict[key] = list(results_dict[key][results_dict[key]['high_error']].iloc[:,0])
    print(results_dict[key][results_dict[key]['high_error']])

                   ENSID      Pred    actual  fold_idx  K562_CAGE_128*3_sum  \
444      ENSG00000177954  3.154718  0.000000         1                  NaN   
779      ENSG00000143106  3.387730  0.000000         1                  NaN   
802      ENSG00000160072  3.078762  0.000000         1                  NaN   
1248     ENSG00000266472  3.169929  0.000000         1                  NaN   
4386     ENSG00000099949  3.149547  0.000000         2                  NaN   
6863     ENSG00000156983  3.267565  0.000000         3                  NaN   
7122     ENSG00000156256  3.190177  0.000000         3                  NaN   
10043    ENSG00000145912  3.236834  0.000000         5                  NaN   
10418    ENSG00000274012  3.116010  0.000000         5                  NaN   
11668    ENSG00000233440  0.127822  3.252559         6              1787.79   
12108  ENSG00000196230_1  0.459130  4.440589         6             27578.70   
12953    ENSG00000181524  0.329853  3.521886        

In [13]:
k5_dict = {}
k5_bool = [gene.decode() in high_error_dict['K562'] for gene in K562_h5['ensid']]
k5_dict['rna_signals'] = K562_h5['rna'][:][k5_bool]
k5_dict['gene_labels'] = K562_h5['ensid'][:][k5_bool]

gm_dict = {}
gm_bool = [gene.decode() in high_error_dict['GM12878'] for gene in K562_h5['ensid']]
gm_dict['rna_signals'] = GM12878_h5['rna'][:][gm_bool]
gm_dict['gene_labels'] = GM12878_h5['ensid'][:][gm_bool]

In [ ]:
for cell_dict in [k5_dict]:
    for i, val in enumerate(cell_dict['gene_labels']):
        plt.plot(cell_dict['rna_signals'][i])
        plt.title(val.decode())
        plt.show()
        plt.clf()
        print(f"Pred: {results_dict['K562'][results_dict['K562'].iloc[:,0] == val.decode()]['Pred'].iloc[0]}")
        print(f"Actual: {results_dict['K562'][results_dict['K562'].iloc[:,0] == val.decode()]['actual'].iloc[0]}")

---
# Examining NA values and their effect on the prediction

In [182]:
drop_NA_results = {}

for key in results_dict.keys():
    print(f'Pearson R for {key} before dropping NA: {pearsonr(results_dict[key]["Pred"], results_dict[key]["actual"])}')
    drop_NA_results[key] = results_dict[key].dropna(subset=f'{key}_CAGE_128*3_sum')
    print(f'Pearson R for {key} after dropping NA: {pearsonr(drop_NA_results[key]["Pred"], drop_NA_results[key]["actual"])}')
    #print(results_dict[key]['Pred'])

Pearson R for K562 before dropping NA: PearsonRResult(statistic=0.7775174261160528, pvalue=0.0)
Pearson R for K562 after dropping NA: PearsonRResult(statistic=0.700872480516171, pvalue=0.0)
Pearson R for GM12878 before dropping NA: PearsonRResult(statistic=0.7994933359171096, pvalue=0.0)
Pearson R for GM12878 after dropping NA: PearsonRResult(statistic=0.744781488751532, pvalue=0.0)


---
# 